# Customer Data Cleanup and Transformation

This notebook:
1. reads the `CustomerExtract.csv` and `CustomerSpec.xlsx` files, 
2. profiles the data,
3. performs data quality checks
4. cleans up data quality issues (for example, removing special characters in the **Name 1** field) 
5. applies any required transformations defined in the spec
5. and outputs cleaned data is exported as a flat file named `CustomerLoad.csv`.

### PLEASE NOTE:
This notebook is created for re-usability based on the Customer Spec excel sheet. Please follow the below steps to produce the appropriate output.


In [2]:
# let us import the libraries/packages we need to conduct the analysis/transformation of the data
import pandas as pd
import numpy as np # may not be needed for this analysis
import re # may not be needed for this analysis
import pydantic # a good package for data validation

pd.set_option('display.max_columns', None)      # Show all columns
pd.set_option('display.max_rows', 50)           # Show all rows (use with caution if your DataFrame is very large)
pd.set_option('display.width', 1000)            # Increase the display width if necessary

# 1. Let's read the data and spec sheet
Note: you can ignore the `UserWarning` below - this isn't applicable for entirety of the notebook.

In [3]:
# Read the customer extract CSV file
df_extract = pd.read_csv('./source/CustomerExtract.csv', encoding='cp1252')

# Read the customer spec Excel file
df_spec = pd.read_excel('./source/CustomerSpec.xlsx', sheet_name='CustomerSpec')

print('CustomerExtract.csv and CustomerSpec.xlsx loaded successfully.')


CustomerExtract.csv and CustomerSpec.xlsx loaded successfully.


d:\Projects\bitbeam\.venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


## 2. Initial Data Profile

We first examine the structure, first few rows, and summary statistics of the extract data. Additionally, if we needed to, we can also grab more details like data skew. Please see below.


In [4]:
# Display basic information about the extract
print('=== Extract DataFrame Info ===')
df_extract.info()

print('\n=== First 5 Rows of Extract ===')
display(df_extract.head())

print('\n=== Summary Statistics for Extract ===')
display(df_extract.describe(include='all'))


=== Extract DataFrame Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 183 entries, KUNNR to ZEORI
dtypes: float64(137), int64(20), object(26)
memory usage: 143.1+ KB

=== First 5 Rows of Extract ===


,KUNNR,LAND1,NAME1,NAME2,ORT01,PSTLZ,REGIO,SORTL,STRAS,TELF1,TELFX,XCPDK,ADRNR,MCOD1,MCOD2,MCOD3,ANRED,AUFSD,BAHNE,BAHNS,BBBNR,BBSNR,BEGRU,BRSCH,BUBKZ,DATLT,ERDAT,ERNAM,EXABL,FAKSD,FISKN,KNAZK,KNRZA,KONZS,KTOKD,KUKLA,LIFNR,LIFSD,LOCCO,LOEVM,NAME3,NAME4,NIELS,ORT02,PFACH,PSTL2,COUNC,CITYC,RPMKR,SPERR,SPRAS,STCD1,STCD2,STKZA,STKZU,TELBX,TELF2,TELTX,TELX1,LZONE,XZEMP,VBUND,STCEG,DEAR1,DEAR2,DEAR3,DEAR4,DEAR5,GFORM,BRAN1,BRAN2,BRAN3,BRAN4,BRAN5,EKONT,UMSAT,UMJAH,UWAER,JMZAH,JMJAH,KATR1,KATR2,KATR3,KATR4,KATR5,KATR6,KATR7,KATR8,KATR9,KATR10,STKZN,UMSA1,TXJCD,PERIV,ABRVW,INSPBYDEBI,INSPATDEBI,KTOCD,PFORT,WERKS,DTAMS,DTAWS,DUEFL,HZUOR,SPERZ,ETIKG,CIVVE,MILVE,KDKG1,KDKG2,KDKG3,KDKG4,KDKG5,XKNZA,FITYP,STCDT,STCD3,STCD4,STCD5,STCD6,XICMS,XXIPI,XSUBT,CFOPC,TXLW1,TXLW2,CCC01,CCC02,CCC03,CCC04,CASSD,KNURL,J_1KFREPRE,J_1KFTBUS,J_1KFTIND,CONFS,UPDAT,UPTIM,NODEL,DEAR6,SUFRAMA,RG,EXP,UF,RGDATE,RIC,RNE,RNEDATE,CNAE,LEGALNAT,CRTN,ICMSTAXPAY,INDTYP,TDT,COMSIZE,DECREGPC,/VSO/R_PALHGT,/VSO/R_PAL_UL,/VSO/R_PK_MAT,/VSO/R_MATPAL,/VSO/R_I_NO_LYR,/VSO/R_ONE_MAT,/VSO/R_ONE_SORT,/VSO/R_ULD_SIDE,/VSO/R_LOAD_PREF,/VSO/R_DPOINT,ALC,PMT_OFFICE,PSOFG,PSOIS,PSON1,PSON2,PSON3,PSOVN,PSOTL,PSOHS,PSOST,PSOO1,PSOO2,PSOO3,PSOO4,PSOO5,ZEORI
0,1,ZZ,"""ZZZ ZZZZÑZZZ, Z.Z.""",NaN,ZZZZZZ,28100,28,ZZZ ZZZZÑZ,"""Z.Z.ZZZZZZZZZZ Z/.ZZZZZZZZZ, 4""",999990990,999999990,NaN,2122,"""ZZZ ZZZZÑZZZ, Z.Z.""",NaN,ZZZZZZ,NaN,NaN,NaN,NaN,8480012,0,NaN,NaN,7,NaN,4/7/2005,ZZZZZZZ,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,10000127.0,NaN,NaN,NaN,NaN,NaN,Z4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZZ,Z28190551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,ZZZ28190551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,0,NaN,NaN,Z,NaN,Z0,Z4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:00:00 ZZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,ZZ,"""ZZZZZZZZZ ZZZZZZ, Z.Z.""",NaN,ZZZZZZZ,1013,1,ZZZZZZZZZ,"""ZZZZZ.ZZZ.ZZZZÑZ- Z/ZZZZZZZZZZ, 8""",999999999,999999999,NaN,1567,"""ZZZZZZZZZ ZZZZZZ, Z.Z.""",NaN,ZZZZZZZ,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,NaN,4/7/2005,ZZZZZZZ,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZZ,Z01015999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,ZZZ01015999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,0,NaN,NaN,Z,NaN,Z0,Z1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:00:00 ZZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,ZZ,"""ZZZZZZZZZZZ ZZZZZZ, Z.Z.""",NaN,ZZZZZZZ ZZZ ZZZZZZ,8210,8,ZZZZZZZZZZ,Z/ZZZZZZZZZZ Zº2 ZZZ.ZZZ.ZZZZZZZ,999999990,999999999,NaN,1744,"""ZZZZZZZZZZZ ZZZZZZ, Z.Z.",NaN,ZZZZZZZ ZZZ ZZZZZZ,NaN,NaN,NaN,NaN,8431236,0,NaN,NaN,7,NaN,4/7/2005,ZZZZZZZ,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,10000182.0,NaN,NaN,NaN,NaN,NaN,Z1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZZ,Z58382516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,ZZZ58382516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,0,NaN,NaN,Z,NaN,Z0,Z2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:00:00 ZZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30,ZZ,"""ZZZZZZ ZZZZZZZZ, Z.Z.""",NaN,ZZZZZZZZZZ,2520,2,ZZZZZZ ZZZ,ZZZ.ZZZZZZZ ZZ.528-Z.Z.ZZZZZZZZZZZ,999999099,999999099,NaN,1786,"""ZZZZZZ ZZZZZZZZ, Z.Z.""",NaN,ZZZZZZZZ,NaN,NaN,NaN,NaN,8481067,0,NaN,NaN,4,NaN,4/7/2005,ZZZZZZZ,NaN,NaN,NaN,NaN,N


=== Summary Statistics for Extract ===


,KUNNR,LAND1,NAME1,NAME2,ORT01,PSTLZ,REGIO,SORTL,STRAS,TELF1,TELFX,XCPDK,ADRNR,MCOD1,MCOD2,MCOD3,ANRED,AUFSD,BAHNE,BAHNS,BBBNR,BBSNR,BEGRU,BRSCH,BUBKZ,DATLT,ERDAT,ERNAM,EXABL,FAKSD,FISKN,KNAZK,KNRZA,KONZS,KTOKD,KUKLA,LIFNR,LIFSD,LOCCO,LOEVM,NAME3,NAME4,NIELS,ORT02,PFACH,PSTL2,COUNC,CITYC,RPMKR,SPERR,SPRAS,STCD1,STCD2,STKZA,STKZU,TELBX,TELF2,TELTX,TELX1,LZONE,XZEMP,VBUND,STCEG,DEAR1,DEAR2,DEAR3,DEAR4,DEAR5,GFORM,BRAN1,BRAN2,BRAN3,BRAN4,BRAN5,EKONT,UMSAT,UMJAH,UWAER,JMZAH,JMJAH,KATR1,KATR2,KATR3,KATR4,KATR5,KATR6,KATR7,KATR8,KATR9,KATR10,STKZN,UMSA1,TXJCD,PERIV,ABRVW,INSPBYDEBI,INSPATDEBI,KTOCD,PFORT,WERKS,DTAMS,DTAWS,DUEFL,HZUOR,SPERZ,ETIKG,CIVVE,MILVE,KDKG1,KDKG2,KDKG3,KDKG4,KDKG5,XKNZA,FITYP,STCDT,STCD3,STCD4,STCD5,STCD6,XICMS,XXIPI,XSUBT,CFOPC,TXLW1,TXLW2,CCC01,CCC02,CCC03,CCC04,CASSD,KNURL,J_1KFREPRE,J_1KFTBUS,J_1KFTIND,CONFS,UPDAT,UPTIM,NODEL,DEAR6,SUFRAMA,RG,EXP,UF,RGDATE,RIC,RNE,RNEDATE,CNAE,LEGALNAT,CRTN,ICMSTAXPAY,INDTYP,TDT,COMSIZE,DECREGPC,/VSO/R_PALHGT,/VSO/R_PAL_UL,/VSO/R_PK_MAT,/VSO/R_MATPAL,/VSO/R_I_NO_LYR,/VSO/R_ONE_MAT,/VSO/R_ONE_SORT,/VSO/R_ULD_SIDE,/VSO/R_LOAD_PREF,/VSO/R_DPOINT,ALC,PMT_OFFICE,PSOFG,PSOIS,PSON1,PSON2,PSON3,PSOVN,PSOTL,PSOHS,PSOST,PSOO1,PSOO2,PSOO3,PSOO4,PSOO5,ZEORI
count,100.000000,100,100,0.0,100,100.00000,100.000000,100,100,80,58,0.0,100.000000,100,0.0,100,2,1.0,0.0,1,1.000000e+02,100.000000,0.0,0.0,100.000000,0.0,100,100,0.0,1.0,0.0,0.0,0.0,0.0,100.00,0.0,4.200000e+01,1.0,0.0,0.0,0.0,0.0,98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100,97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100,0.0,0.0,97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,100.0,100.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,0.0,98.0,0.0,0.0,0.0,1,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100,100.0,0.0,0.0,100,0.0,98,98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
unique,NaN,1,90,NaN,50,NaN,NaN,27,100,24,17,NaN,NaN,87,NaN,50,2,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN,1,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,ZZ,"""ZZZZZZ, Z.Z.""",NaN,ZZZZZZZZ,NaN,NaN,ZZZZZZZZZ,"""Z.Z.ZZZZZZZZZZ Z/.ZZZZZZZZZ, 4""",999999999,999999999,NaN,NaN,"""ZZZZZZ, Z.Z.""",NaN,ZZZZZZZZ,ZZZZZZZ,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,4/7/2005,ZZZZZZZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZZ,Z28425270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,ZZZ28425270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,NaN,NaN,NaN,Z,NaN,Z0,Z1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,NaN,NaN,NaN,NaN,NaN,NaN,12:00:00 ZZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,100,5,NaN,14,NaN,NaN,22,1,35,24,NaN,NaN,5,NaN,15,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,97,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,100,NaN,98,53,NaN,NaN,NaN,NaN,

### Data Skew Analysis

In this section, we analyze the skewness of numeric columns in the extract to identify any significant asymmetries in the data distribution. A skewness value near 0 indicates a relatively symmetric distribution, while positive or negative values indicate right or left skew, respectively.

For categorical columns, we can also check the distribution using value counts.

**CAUTION** --> RUN AT YOUR OWN RISK!

This will produce an extended output detailing each column and skewed distributions (or imbalances for that matter).

In [ ]:
# Data Skew Analysis for Numeric Columns
numeric_columns = df_extract.select_dtypes(include=['number']).columns
if len(numeric_columns) > 0:
    skewness = df_extract[numeric_columns].skew()
    print("Skewness of numeric columns:")
    display(skewness)
else:
    print("No numeric columns found for skewness analysis.")

# Optional: Analyze distribution for categorical columns
categorical_columns = df_extract.select_dtypes(include=['object']).columns
print("\nFrequency distribution for categorical columns:")
for col in categorical_columns:
    print(f"\nDistribution for {col}:")
    display(df_extract[col].value_counts(dropna=False))


# Post-Profile Analysis

After profiling the data and comparing to the spec sheet, we notice a few columns don't exist in the extract but should exist in the new format with appropriate value mapping:
1. BUKRS (Use Company Code G100)
2. VKORG (Use Sales Organization G100)
3. VTWEG (Use Existing Distribution Channel 20)
4. SPART (Use Existing Division 10)

These exist from a different SAP Table with appropriate mapping given by the spec sheet transformation logic. Expect these to be added from table `RF02D` with applied logic later on in step 5.


## 3. Data Quality Checks

In this section we:
- Identify any fields in the extract that are not present in the spec.
- Identify fields that are in the spec but are not flagged as utilized in the legacy system (based on the flag in the spec) yet are populated with data.

The Spec file should include **SAP FIELD** and a flag column **Field Utilized in LEGACY System**

In [6]:
# Prepare the spec DataFrame for analysis
df_spec['Field Utilized in LEGACY System'] = df_spec['Field Utilized in LEGACY System'].astype(str).str.upper()

# List of fields defined in the spec
spec_fields = df_spec['SAP FIELD'].tolist()

# List of fields flagged as utilized in the legacy system
legacy_fields = df_spec.loc[df_spec['Field Utilized in LEGACY System'].isin(['YES', 'Y', 'TRUE']), 'SAP FIELD'].tolist()

# Identify columns in the extract that are not defined in the spec
extra_columns_not_in_spec = [col for col in df_extract.columns if col not in spec_fields]

# Identify columns that are in the spec but are NOT flagged as utilized in the legacy system
non_legacy_fields_in_spec = [col for col in df_extract.columns if col in spec_fields and col not in legacy_fields]

print('--- Quality Check Results BELOW ---\n')

print('Columns in the extract NOT found in the spec sheet:')
print(extra_columns_not_in_spec if extra_columns_not_in_spec else 'None')

print('\nColumns in the extract that are in the spec sheet but NOT flagged as utilized in the legacy system:')
print(non_legacy_fields_in_spec if non_legacy_fields_in_spec else 'None')

# For traceability, display non-null counts for these columns
if extra_columns_not_in_spec:
    print('\nNon-null counts for columns not in spec:')
    display(df_extract[extra_columns_not_in_spec].count())
else:
    print('\nNo extra columns found in extract that are missing in spec.')

if non_legacy_fields_in_spec:
    print('\nNon-null counts for columns in spec but not flagged as legacy:')
    display(df_extract[non_legacy_fields_in_spec].count())
else:
    print('\nAll columns in extract that are in the spec are flagged as legacy (or none are populated).')


--- Quality Check Results BELOW ---

Columns in the extract NOT found in the spec sheet:
['STCD6', 'SUFRAMA', 'RG', 'EXP', 'UF', 'RGDATE', 'RIC', 'RNE', 'RNEDATE', 'CNAE', 'LEGALNAT', 'CRTN', 'ICMSTAXPAY', 'INDTYP', 'TDT', 'COMSIZE', 'DECREGPC', 'ZEORI']

Columns in the extract that are in the spec sheet but NOT flagged as utilized in the legacy system:
['TELFX', 'XCPDK', 'ADRNR', 'MCOD1', 'MCOD2', 'MCOD3', 'ANRED', 'AUFSD', 'BAHNE', 'BAHNS', 'BBBNR', 'BBSNR', 'BEGRU', 'BRSCH', 'BUBKZ', 'DATLT', 'EXABL', 'FAKSD', 'FISKN', 'KNAZK', 'KNRZA', 'KONZS', 'KUKLA', 'LOCCO', 'NAME3', 'NAME4', 'NIELS', 'ORT02', 'PFACH', 'PSTL2', 'RPMKR', 'STKZA', 'STKZU', 'TELBX', 'TELF2', 'TELTX', 'TELX1', 'DEAR1', 'DEAR2', 'DEAR3', 'DEAR4', 'DEAR5', 'GFORM', 'BRAN1', 'BRAN2', 'BRAN3', 'BRAN4', 'BRAN5', 'EKONT', 'UMSAT', 'UMJAH', 'UWAER', 'JMZAH', 'JMJAH', 'KATR1', 'KATR3', 'KATR4', 'KATR5', 'KATR6', 'KATR7', 'KATR10', 'STKZN', 'UMSA1', 'PERIV', 'ABRVW', 'INSPBYDEBI', 'INSPATDEBI', 'KTOCD', 'PFORT', 'WERKS', 'D

STCD6           0
SUFRAMA         0
RG              0
EXP             0
UF              0
RGDATE          0
RIC           100
RNE             0
RNEDATE         0
CNAE            0
LEGALNAT      100
CRTN            0
ICMSTAXPAY      0
INDTYP          0
TDT             0
COMSIZE         0
DECREGPC        0
ZEORI           0
dtype: int64


Non-null counts for columns in spec but not flagged as legacy:


TELFX     58
XCPDK      0
ADRNR    100
MCOD1    100
MCOD2      0
        ... 
PSOO1      0
PSOO2      0
PSOO3      0
PSOO4      0
PSOO5      0
Length: 135, dtype: int64

## 4. Data Cleanup
In this section, we perform data cleanup and apply the necessary transformations as defined in the spec. As an example, we clean up the **Name 1** field by removing any special characters and extra whitespace. 

You can extend these transformations as needed based on the spec requirements (for example, converting dates to a standardized format, trimming whitespace across all text columns, etc.).


In [7]:
# Clean columns fields by removing special characters
cleanse_columns = [
    'NAME1',
    'ORT01',
    'SORTL',
    'STRAS',
    'MCOD1', # not mapped in spec but present in extract
    'MCOD3' # not mapped in spec but present in extract
]

for col in cleanse_columns:
    if col in df_extract.columns:
        # Remove any character that is not alphanumeric or whitespace and then strip leading/trailing whitespace
        df_extract[col] = df_extract[col].astype(str).str.replace(r'[^a-zA-Z0-9\s]', '', regex=True).str.strip()

    # Additional Transformation: Trim whitespace in all string columns
    str_cols = df_extract.select_dtypes(include='object').columns
    for col in str_cols:
        df_extract[col] = df_extract[col].str.strip()

    # Placeholder for additional transformations as specified in the spec (sorry, just psuedo-coding here)
    # For example: date formatting, field renaming, type conversions, etc maybe I'll add some later

print('Data cleanup and transformations applied successfully.')
display(df_extract.head(100))


Data cleanup and transformations applied successfully.


,KUNNR,LAND1,NAME1,NAME2,ORT01,PSTLZ,REGIO,SORTL,STRAS,TELF1,TELFX,XCPDK,ADRNR,MCOD1,MCOD2,MCOD3,ANRED,AUFSD,BAHNE,BAHNS,BBBNR,BBSNR,BEGRU,BRSCH,BUBKZ,DATLT,ERDAT,ERNAM,EXABL,FAKSD,FISKN,KNAZK,KNRZA,KONZS,KTOKD,KUKLA,LIFNR,LIFSD,LOCCO,LOEVM,NAME3,NAME4,NIELS,ORT02,PFACH,PSTL2,COUNC,CITYC,RPMKR,SPERR,SPRAS,STCD1,STCD2,STKZA,STKZU,TELBX,TELF2,TELTX,TELX1,LZONE,XZEMP,VBUND,STCEG,DEAR1,DEAR2,DEAR3,DEAR4,DEAR5,GFORM,BRAN1,BRAN2,BRAN3,BRAN4,BRAN5,EKONT,UMSAT,UMJAH,UWAER,JMZAH,JMJAH,KATR1,KATR2,KATR3,KATR4,KATR5,KATR6,KATR7,KATR8,KATR9,KATR10,STKZN,UMSA1,TXJCD,PERIV,ABRVW,INSPBYDEBI,INSPATDEBI,KTOCD,PFORT,WERKS,DTAMS,DTAWS,DUEFL,HZUOR,SPERZ,ETIKG,CIVVE,MILVE,KDKG1,KDKG2,KDKG3,KDKG4,KDKG5,XKNZA,FITYP,STCDT,STCD3,STCD4,STCD5,STCD6,XICMS,XXIPI,XSUBT,CFOPC,TXLW1,TXLW2,CCC01,CCC02,CCC03,CCC04,CASSD,KNURL,J_1KFREPRE,J_1KFTBUS,J_1KFTIND,CONFS,UPDAT,UPTIM,NODEL,DEAR6,SUFRAMA,RG,EXP,UF,RGDATE,RIC,RNE,RNEDATE,CNAE,LEGALNAT,CRTN,ICMSTAXPAY,INDTYP,TDT,COMSIZE,DECREGPC,/VSO/R_PALHGT,/VSO/R_PAL_UL,/VSO/R_PK_MAT,/VSO/R_MATPAL,/VSO/R_I_NO_LYR,/VSO/R_ONE_MAT,/VSO/R_ONE_SORT,/VSO/R_ULD_SIDE,/VSO/R_LOAD_PREF,/VSO/R_DPOINT,ALC,PMT_OFFICE,PSOFG,PSOIS,PSON1,PSON2,PSON3,PSOVN,PSOTL,PSOHS,PSOST,PSOO1,PSOO2,PSOO3,PSOO4,PSOO5,ZEORI
0,1,ZZ,ZZZ ZZZZZZZ ZZ,NaN,ZZZZZZ,28100,28,ZZZ ZZZZZ,ZZZZZZZZZZZZ ZZZZZZZZZZ 4,999990990,999999990,NaN,2122,ZZZ ZZZZZZZ ZZ,NaN,ZZZZZZ,NaN,NaN,NaN,NaN,8480012,0,NaN,NaN,7,NaN,4/7/2005,ZZZZZZZ,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,10000127.0,NaN,NaN,NaN,NaN,NaN,Z4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZZ,Z28190551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,ZZZ28190551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,0,NaN,NaN,Z,NaN,Z0,Z4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:00:00 ZZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,ZZ,ZZZZZZZZZ ZZZZZZ ZZ,NaN,ZZZZZZZ,1013,1,ZZZZZZZZZ,ZZZZZZZZZZZZZ ZZZZZZZZZZZ 8,999999999,999999999,NaN,1567,ZZZZZZZZZ ZZZZZZ ZZ,NaN,ZZZZZZZ,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,NaN,4/7/2005,ZZZZZZZ,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZZ,Z01015999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,ZZZ01015999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,0,NaN,NaN,Z,NaN,Z0,Z1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:00:00 ZZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,ZZ,ZZZZZZZZZZZ ZZZZZZ ZZ,NaN,ZZZZZZZ ZZZ ZZZZZZ,8210,8,ZZZZZZZZZZ,ZZZZZZZZZZZ Z2 ZZZZZZZZZZZZZ,999999990,999999999,NaN,1744,ZZZZZZZZZZZ ZZZZZZ ZZ,NaN,ZZZZZZZ ZZZ ZZZZZZ,NaN,NaN,NaN,NaN,8431236,0,NaN,NaN,7,NaN,4/7/2005,ZZZZZZZ,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,10000182.0,NaN,NaN,NaN,NaN,NaN,Z1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZZ,Z58382516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,ZZZ58382516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,0,NaN,NaN,Z,NaN,Z0,Z2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:00:00 ZZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30,ZZ,ZZZZZZ ZZZZZZZZ ZZ,NaN,ZZZZZZZZZZ,2520,2,ZZZZZZ ZZZ,ZZZZZZZZZZ ZZ528ZZZZZZZZZZZZZ,999999099,999999099,NaN,1786,ZZZZZZ ZZZZZZZZ ZZ,NaN,ZZZZZZZZ,NaN,NaN,NaN,NaN,8481067,0,NaN,NaN,4,NaN,4/7/2005,ZZZZZZZ,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,10000323.0,NaN,NaN,NaN,NaN,NaN,Z2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZZ,Z02001980,NaN,NaN,NaN,NaN,N

### Verify Postal Code Column

In [8]:
# Convert PSTLZ and REGIO to strings and remove any extra whitespace
df_extract['PSTLZ'] = df_extract['PSTLZ'].astype(str).str.strip()
df_extract['REGIO'] = df_extract['REGIO'].astype(str).str.strip()

# Identify rows where PSTLZ does not start with REGIO
invalid_rows = df_extract[~df_extract.apply(lambda row: row['PSTLZ'].startswith(row['REGIO']), axis=1)]

# Report results
print(f"Number of rows where PSTLZ does not start with REGIO: {len(invalid_rows)}")
if not invalid_rows.empty:
    print("Sample rows with mismatched PSTLZ and REGIO:")
    display(invalid_rows[['PSTLZ', 'REGIO']].head())
else:
    print("All rows have PSTLZ starting with REGIO.")


Number of rows where PSTLZ does not start with REGIO: 0
All rows have PSTLZ starting with REGIO.


### Verify the Date Column (ERDAT)

In [13]:
# Convert ERDAT column to datetime using the expected format (mm/dd/yyyy)
df_extract['ERDAT_converted'] = pd.to_datetime(df_extract['ERDAT'], format='%m/%d/%Y', errors='coerce')

# Identify rows where conversion failed (i.e., invalid date format)
invalid_dates = df_extract[df_extract['ERDAT_converted'].isna() & df_extract['ERDAT'].notna()]

print(f"Number of rows with invalid ERDAT dates: {len(invalid_dates)}")
if not invalid_dates.empty:
    print("Sample rows with invalid ERDAT values:")
    display(invalid_dates[['ERDAT']].head())
else:
    print("All ERDAT values are valid and match the format mm/dd/yyyy.")


Number of rows with invalid ERDAT dates: 0
All ERDAT values are valid and match the format mm/dd/yyyy.


# 5. Transformations

#### Added columns based on spec sheet, check the above comment regarding added columns

In [9]:
# Add columns from RF02D as specified in the spec sheet

# According to the spec sheet:
# 1. BUKRS: Use Company Code G100
# 2. VKORG: Use Sales Organization G100
# 3. VTWEG: Use Existing Distribution Channel 20
# 4. SPART: Use Existing Division 10

df_extract['BUKRS'] = 'G100'
df_extract['VKORG'] = 'G100'
df_extract['VTWEG'] = '20'
df_extract['SPART'] = '10'

print('Added RF02D columns:')
display(df_extract[['BUKRS', 'VKORG', 'VTWEG', 'SPART']].head())

Added RF02D columns:


,BUKRS,VKORG,VTWEG,SPART
0,G100,G100,20,10
1,G100,G100,20,10
2,G100,G100,20,10
3,G100,G100,20,10
4,G100,G100,20,10


#### Handle the LIFNR column (account number)

The LIFNR column is an account number that shouldn't be a decimal column. Specifically, it should be an int-based column so we handle that conversion.

In [10]:
# Convert LIFNR column to an integer type using a nullable integer (Int64)
df_extract['LIFNR'] = pd.to_numeric(df_extract['LIFNR'], errors='coerce').astype('Int64')

print("LIFNR column converted to integer type:")
display(df_extract['LIFNR'].head())

LIFNR column converted to integer type:


0    10000127
1        <NA>
2    10000182
3    10000323
4        <NA>
Name: LIFNR, dtype: Int64

#### Filtering Output Columns

According to the spec sheet, only include fields that are marked as **"Y"** in the **Field Used** column. The following cell filters the DataFrame so that the final output only contains those required fields.


In [11]:
# Process the "Field Used" column from the spec to ensure consistency
df_spec['Field Used'] = df_spec['Field Used'].astype(str).str.upper()

# Create a list of fields to output based on the spec where Field Used is "Y"
fields_to_output = df_spec.loc[df_spec['Field Used'] == 'Y', 'SAP FIELD'].tolist()

# Ensure that we only include fields that are present in our extract DataFrame
fields_to_output = [col for col in fields_to_output if col in df_extract.columns]

print("Fields to output (based on spec):")
print(fields_to_output)

# Filter the extract DataFrame to only include the desired columns
df_extract = df_extract[fields_to_output]

print("DataFrame after filtering to include only fields marked as 'Y':")
display(df_extract.head())


Fields to output (based on spec):
['KTOKD', 'KUNNR', 'BUKRS', 'VKORG', 'VTWEG', 'SPART', 'LAND1', 'NAME1', 'NAME2', 'ORT01', 'PSTLZ', 'REGIO', 'SORTL', 'STRAS', 'TELF1', 'AUFSD', 'ERDAT', 'ERNAM', 'LIFNR', 'LIFSD', 'LOEVM', 'NAME3', 'NAME4', 'COUNC', 'CITYC', 'SPERR', 'SPRAS', 'STCD1', 'STCD2', 'LZONE', 'XZEMP', 'VBUND', 'STCEG', 'KATR1', 'KATR2', 'KATR3', 'KATR4', 'KATR5', 'KATR6', 'KATR7', 'KATR8', 'KATR9', 'KATR10', 'TXJCD', 'KDKG1', 'KDKG2', 'KDKG3', 'KDKG4', 'KDKG5', 'STCD5']
DataFrame after filtering to include only fields marked as 'Y':


,KTOKD,KUNNR,BUKRS,VKORG,VTWEG,SPART,LAND1,NAME1,NAME2,ORT01,PSTLZ,REGIO,SORTL,STRAS,TELF1,AUFSD,ERDAT,ERNAM,LIFNR,LIFSD,LOEVM,NAME3,NAME4,COUNC,CITYC,SPERR,SPRAS,STCD1,STCD2,LZONE,XZEMP,VBUND,STCEG,KATR1,KATR2,KATR3,KATR4,KATR5,KATR6,KATR7,KATR8,KATR9,KATR10,TXJCD,KDKG1,KDKG2,KDKG3,KDKG4,KDKG5,STCD5
0,1,1,G100,G100,20,10,ZZ,ZZZ ZZZZZZZ ZZ,NaN,ZZZZZZ,28100,28,ZZZ ZZZZZ,ZZZZZZZZZZZZ ZZZZZZZZZZ 4,999990990,NaN,4/7/2005,ZZZZZZZ,10000127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZZ,Z28190551,NaN,1,NaN,NaN,ZZZ28190551,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,Z0,Z4,NaN,NaN,NaN,NaN
1,1,3,G100,G100,20,10,ZZ,ZZZZZZZZZ ZZZZZZ ZZ,NaN,ZZZZZZZ,1013,1,ZZZZZZZZZ,ZZZZZZZZZZZZZ ZZZZZZZZZZZ 8,999999999,NaN,4/7/2005,ZZZZZZZ,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZZ,Z01015999,NaN,1,NaN,NaN,ZZZ01015999,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,Z0,Z1,NaN,NaN,NaN,NaN
2,1,7,G100,G100,20,10,ZZ,ZZZZZZZZZZZ ZZZZZZ ZZ,NaN,ZZZZZZZ ZZZ ZZZZZZ,8210,8,ZZZZZZZZZZ,ZZZZZZZZZZZ Z2 ZZZZZZZZZZZZZ,999999990,NaN,4/7/2005,ZZZZZZZ,10000182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZZ,Z58382516,NaN,1,NaN,NaN,ZZZ58382516,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,Z0,Z2,NaN,NaN,NaN,NaN
3,1,30,G100,G100,20,10,ZZ,ZZZZZZ ZZZZZZZZ ZZ,NaN,ZZZZZZZZZZ,2520,2,ZZZZZZ ZZZ,ZZZZZZZZZZ ZZ528ZZZZZZZZZZZZZ,999999099,NaN,4/7/2005,ZZZZZZZ,10000323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZZ,Z02001980,NaN,1,NaN,NaN,ZZZ02001980,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,Z0,Z1,NaN,NaN,NaN,NaN
4,1,31,G100,G100,20,10,ZZ,ZZZZ ZZ,NaN,ZZZZZZZZ,31001,31,ZZZZ ZZ,ZZZZZZZZZZ 3,NaN,NaN,4/9/2005,ZZZZZZ,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZZ,Z31072440,NaN,1,NaN,NaN,ZZZ31072440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 6. Exporting the Transformed Data

The final cleaned and transformed data is saved as a flat file named `CustomerLoad.csv`.


In [12]:
# Write the transformed DataFrame to a CSV file
output_file = './target/CustomerLoad.csv'
df_extract.to_csv(output_file, index=False)
print(f'File saved as {output_file}')


File saved as ./target/CustomerLoad.csv


# Additional Details/Fun stuff we can do based off of spec sheet

We mentioned earlier we could calculate data skew (for processing purposes) but we can also do other things based off of the spec sheet selection/transformation logic.

1. Since were migrating to a new system we may want to implement new customer identifier logic. Let's say this company "ABC" belongs to a parent customer "CBA". The CBA company owns several companies and would like to identify each sub-company customer by a unique idenifier. This sub company - "ABC" - will be prefixed in this manner.
2. There was is a SAP FIELD named KUNNR (for customer/customer code), could be used for a join.
3. The postal code column can be treated with specific logic to ensure the quality of the data inputted. Since were working on a Country + Region coluimn - we can ensure the region is prefixed in the PSTLZ column. Logic to determine what format the PSTLZ should be can be added too (US vs non-US). Additionally, you can use a python package called **uszipcode** to further verify the postal code column to ensure it is valid.
4. This is a perfect case to the **Pydantic package for data validation**. This is widely-accepted package for ensuring data adheres (and also coerce) to a specific BaseModel.
5. According to the spec sheet, there is 2 different SAP tables that are involved. I chose to not output two files because the instructions insisted on one CSV file.